# TO DO :
## 1) Check Bigrams
## 2) Check on Brand Name scraping
## 3) Check if voting increases accuracy
## 4) tune classifier

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import grid_search
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn import cross_validation

stop = stopwords.words('english')
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 

In [2]:
df  = pd.read_table('classification_train.tsv',names=['title','brand_id','category_id'])

In [4]:
#df.category_id.value_counts().plot(kind='bar')

In [5]:
#list(df.title[:10])

In [24]:
import re
reg = r'[a-zA-Z]{2,}'
reg = re.compile(reg,re.VERBOSE|re.IGNORECASE)
#txt = reg.findall('DIGI CM 16 PORT RJ-45 CONSOLE SERVER - Model#: 70001910')
noise =  ['years','warranty','gb','high','mm','ac','ft','protective','us','white','blue','tm','includes','formatted','mb',
               'shoulder','pink','page','support','free','animal','non','dc','switch','yellow','mat','super','brother'
              ,'lg','nr','tek','savvv','mbps','ca','adjustable','made']
stop = stop + noise

def preprocess(txt,reg,stop=stop):
    txt = nltk.tokenize.word_tokenize(txt)
    txt = nltk.pos_tag(txt)
    t = []
    for i in txt:
        if i[1]=='NNP' or i[1]=='NN':
            t.append(i[0])
    t = reg.findall(" ".join(t))
    txt1 = [] 
    for i in t:
        if i.lower() not in stop and not i.startswith(tuple('123456789')):
            txt1.append(i.lower())
    return " ".join(txt1)

#test
#preprocess(list(df.title[:1])[0],reg)
df.shape

(999987, 3)

In [2]:
preprocessor = lambda x:preprocess(x,reg)
lim = df.shape[0]
df.title = df.title.apply(preprocessor)
df.to_csv("preprocessed.csv")

NameError: name 'df' is not defined

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
df.tail(40)
df = df.fillna('nan')
ind = df[df.brand_id=='bid'].index
df = df.drop(df.index[ind])

In [3]:
all_classes = np.array(df.brand_id.drop_duplicates(),dtype=int)

In [118]:
"""from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#min_df = 1000
count_vect = CountVectorizer(analyzer = "word",min_df=2000)
X_train= count_vect.fit_transform(list(df.title[:100000]))
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train)
X_train_tf = tf_transformer.transform(X_train).toarray()
final_feature_set  = pd.DataFrame(X_train_tf)
final_feature_set['category_id'] = df.category_id[:100000]
final_feature_set = np.asarray(final_feature_set)
label_set = np.asarray(list(df.brand_id[:100000]))
#final_feature_set
print(final_feature_set.shape)
count_vect.get_feature_names()"""

'from sklearn.feature_extraction.text import CountVectorizer\nfrom sklearn.feature_extraction.text import TfidfTransformer\n#min_df = 1000\ncount_vect = CountVectorizer(analyzer = "word",min_df=2000)\nX_train= count_vect.fit_transform(list(df.title[:100000]))\ntf_transformer = TfidfTransformer(use_idf=False).fit(X_train)\nX_train_tf = tf_transformer.transform(X_train).toarray()\nfinal_feature_set  = pd.DataFrame(X_train_tf)\nfinal_feature_set[\'category_id\'] = df.category_id[:100000]\nfinal_feature_set = np.asarray(final_feature_set)\nlabel_set = np.asarray(list(df.brand_id[:100000]))\n#final_feature_set\nprint(final_feature_set.shape)\ncount_vect.get_feature_names()'

In [6]:
#final_feature_set

In [7]:
### View most frequent words
"""freq = list(zip(count_vect.get_feature_names(),
    np.asarray(X.sum(axis=0)).ravel()))

freq = list(reversed(sorted(freq, key=lambda x: x[1])))"""

'freq = list(zip(count_vect.get_feature_names(),\n    np.asarray(X.sum(axis=0)).ravel()))\n\nfreq = list(reversed(sorted(freq, key=lambda x: x[1])))'

In [41]:
""""print("Declaring Classifiers")
clf1 = LogisticRegression(random_state=1)
print("Declaring Classifiers")
clf2 = RandomForestClassifier(random_state=1)
print("Declaring Classifiers")
clf3 = SVC(kernel='linear')
print("Declaring Classifiers")
clf4 = SGDClassifier()
print("Declaring Classifiers")
clf5 = MultinomialNB()
print("Declaring Classifiers")
#('lr', clf1),('SGCD',clf4),('mNB',clf5)
v_clf = VotingClassifier(estimators=[ ('rf', clf2), ('svc', clf3)], voting='soft')
print("Declaring Classifiers")


print("looping now")
for clf, label in zip([clf2, clf3, v_clf], ['Random Forest', 'Support Vector Machine','Voting']):
    print("working on ",label)
    scores = cross_validation.cross_val_score(clf, final_feature_set,label_set, cv=5, scoring='accuracy',verbose=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))""""

    
"""for i, (X_train_text, y_train) in enumerate(minibatch_iterators):

    tick = time.time()
    X_train = vectorizer.transform(X_train_text)
    total_vect_time += time.time() - tick

    for cls_name, cls in partial_fit_classifiers.items():
        tick = time.time()
        # update estimator with examples in the current mini-batch
        cls.partial_fit(X_train, y_train, classes=all_classes)
        print(cls.score(X_test, y_test))"""

SyntaxError: EOL while scanning string literal (<ipython-input-41-71b2ee39d8b9>, line 21)

In [9]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import StratifiedKFold
import time 
vec = HashingVectorizer(decode_error='ignore', n_features=3000,
                               non_negative=True)



In [9]:
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
partial_fit_classifiers = {
    #'SGD': SGDClassifier(),
    'Perceptron': Perceptron(),
   'NB Multinomial': MultinomialNB(alpha=0.01),
   'Passive-Aggressive': PassiveAggressiveClassifier(),
}

In [11]:
def make_batch(batch_size,split,df,vec):
    df_test = df[split:split+batch_size]
    split = split+batch_size+1
    try:
        X_train = vec.transform(list(df_test.title))
    except(ValueError):
        print(split," ",split+batch_size)
    #X_train = pd.DataFrame(X_train)
    #X_train['category_id'] = df_test.category_id
    #X_train = np.asarray(X_train)
    y_train  = np.asarray(df_test.brand_id,dtype=int)
    return X_train,y_train,split


X_test,y_test,split = make_batch(1987,0,df,vec)

for i in range(0,998):
    X_train, y_train , split = make_batch(1000,split,df,vec)
    main_tick = time.time()
    for cls_name, cls in partial_fit_classifiers.items():
        tick = time.time()
        # update estimator with examples in the current mini-batch
        cls.partial_fit(X_train, y_train, classes=all_classes)
        print("Iteration",i)
        print("Classifier ",cls_name,"-Accuracy: ",cls.score(X_test, y_test))
        print("Execution Time: ",time.time() - tick)
        
        
total_time_taken = time.time() - main_tick


   
print("Total_Time_for_Execution: ",total_time_taken)

Iteration 0
Classifier  SGD -Accuracy:  0.228485153498
Execution Time:  26.602406978607178
Iteration 1
Classifier  SGD -Accuracy:  0.27277302466
Execution Time:  26.595905780792236
Iteration 2
Classifier  SGD -Accuracy:  0.339204831404
Execution Time:  11.714334726333618
Iteration 3
Classifier  SGD -Accuracy:  0.347760442879
Execution Time:  10.867953777313232
Iteration 4
Classifier  SGD -Accuracy:  0.370910920986
Execution Time:  13.746166229248047
Iteration 5
Classifier  SGD -Accuracy:  0.380473074987
Execution Time:  10.249488592147827
Iteration 6
Classifier  SGD -Accuracy:  0.391545042778
Execution Time:  9.904654741287231
Iteration 7
Classifier  SGD -Accuracy:  0.410166079517
Execution Time:  9.761350631713867
Iteration 8
Classifier  SGD -Accuracy:  0.428283844992
Execution Time:  9.95571231842041
Iteration 9
Classifier  SGD -Accuracy:  0.442878711626
Execution Time:  10.393485307693481
Iteration 10
Classifier  SGD -Accuracy:  0.4514343231
Execution Time:  9.925247192382812
Iterat

In [1]:
from sklearn.externals import joblib
gnb = joblib.load('SGDC.pkl')

In [21]:
df_blind = pd.read_table('classification_blind_set_corrected.tsv',names=['title','category_id'])

In [25]:
preprocessor = lambda x:preprocess(x,reg)
df_blind.title = df_blind.title.apply(preprocessor)
df_blind.to_csv("preprocessed_test.csv")

In [4]:
df_blind = pd.read_csv('preprocessed_test.csv')

In [5]:
df_blind = df_blind.drop('Unnamed: 0',axis=1)


In [6]:
df_blind = df_blind.fillna('nan')


In [7]:
ind = df_blind[df_blind.title=='title'].index
df_blind = df_blind.drop(df_blind.index[ind])

In [13]:
df_blind.shape[0] - (1000*619)
X ,split= make_batch1(df_blind.shape[0],0,df_blind,vec)


In [28]:
"""split = 0
submit = []
for i in range(0,1238):
    print("Iteration: ",i)
    l = list(gnb.predict(X[split:split+500]))
    print(l[:10])
    for label in l:
        submit.append(label)
    print("Done")
    split = split + 500 +1
"""
#df_blind.shape[0]-len(submit)
l = list(gnb.predict(X[len(submit)+1:]))
print(l[:10])
for label in l:
    submit.append(label)
        


[20313, 19709, 34996, 26992, 19709, 33613, 35038, 28421, 23271, 40943]


619238

In [61]:
l= list(gnb.predict(X[-1]))




In [63]:
submit.append(l)


In [65]:
with open('submit.txt','w') as f:
    for s in submit:
        f.write(str(s)+"\n")


In [11]:
def make_batch1(batch_size,split,df,vec):
    df_test = df[split:split+batch_size]
    split = split+batch_size+1
    try:
        X_train = vec.transform(list(df_test.title))
    except(ValueError):
        print(split," ",split+batch_size)
    return X_train,split
    

In [13]:
"""def make_batch(batch_size,split,df,vec):
    df_test = df[split:split+batch_size]
    split = split+batch_size+1
    try:
        X_train = vec.transform(list(df_test.title))
    except(ValueError):
        print(split," ",split+batch_size)
    #X_train = pd.DataFrame(X_train)
    #X_train['category_id'] = df_test.category_id
    #X_train = np.asarray(X_train)
    y_train  = np.asarray(df_test.brand_id,dtype=int)
    return X_train,y_train,split


X_test,y_test,split = make_batch(1987,0,df,vec)

for i in range(0,998):
    X_train, y_train , split = make_batch(1000,split,df,vec)
    main_tick = time.time()
    for cls_name, cls in partial_fit_classifiers.items():
        tick = time.time()
        # update estimator with examples in the current mini-batch
        cls.partial_fit(X_train, y_train, classes=all_classes)
        print("Iteration",i)
        print("Classifier ",cls_name,"-Accuracy: ",cls.score(X_test, y_test))
        print("Execution Time: ",time.time() - tick)
        
        
total_time_taken = time.time() - main_tick
   
print("Total_Time_for_Execution: ",total_time_taken)"""

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt
